## Postprocessing
> Postprocessing step to cleanup elimination events.

In [1]:
#| default_exp segmentation.postprocessing
#| export
import dataclasses

from csgo_clips_autotrim.segmentation.elimination import EliminationSegmentationResult, is_bbox_overlap, bbox_area

In [5]:
#| export
def remove_duplicate_events(result: EliminationSegmentationResult) -> EliminationSegmentationResult:
    """Remove duplicate elimination events (overlapping event bboxes) over the
    same region.

    Args:
        result (EliminationSegmentationResult): input result

    Returns:
        EliminationSegmentationResult
    """
    deduped_events = []

    for i, event1 in enumerate(result.elimination_events):
        for event2 in result.elimination_events[i+1:]:
            if is_bbox_overlap(event1.event.bbox, event2.event.bbox):
                bigger_event = event1

                if bbox_area(event1.event.bbox) < bbox_area(event2.event.bbox):
                    bigger_event = event2
                
                deduped_events.append(bigger_event)
    
    return dataclasses.replace(result, elimination_events=deduped_events)

### Test run

In [8]:
from csgo_clips_autotrim.experiment_utils.constants import BASE_DIR


sample_result_path =  BASE_DIR / 'nbs' / 'out' / 'work-dir' / 'provenance' / 'd2_1v4_ace_00006.json'
sample_result = EliminationSegmentationResult.from_json(sample_result_path.read_text())
sample_result

EliminationSegmentationResult(elimination_events=[EliminationEvent(event=SegmentationResult(bbox=XYXYBBox(dims=[320.96661376953125, 36.627685546875, 584.9686889648438, 54.144866943359375]), ocr=None), eliminator=SegmentationResult(bbox=XYXYBBox(dims=[320.96661376953125, 40.964388847351074, 434.2342987060547, 52.10789680480957]), ocr=OCRResult(text='RANOUAPEEK Hiccup', confidence=0.9002308249473572)), weapon=SegmentationResult(bbox=XYXYBBox(dims=[434.2342987060547, 40.964388847351074, 475.1927185058594, 52.10789680480957]), ocr=None), eliminated=SegmentationResult(bbox=XYXYBBox(dims=[475.1927185058594, 40.964388847351074, 584.9686889648438, 52.10789680480957]), ocr=OCRResult(text='Hutch wala kutta', confidence=0.9410244226455688))), EliminationEvent(event=SegmentationResult(bbox=XYXYBBox(dims=[320.46221923828125, 37.169036865234375, 501.333251953125, 53.19525146484375]), ocr=None), eliminator=SegmentationResult(bbox=XYXYBBox(dims=[320.46221923828125, 40.94595241546631, 434.4218368530273

In [9]:
print('Original events: ', len(sample_result.elimination_events))
deduped_result = remove_duplicate_events(sample_result)
print('Deduped events: ', len(deduped_result.elimination_events))

Original events:  2
Deduped events:  1
